In [ ]:
import os
from tqdm import tqdm

In [ ]:
from avapi.carla import CarlaScenesManager


# cpath = os.path.join("/data/shared/CARLA/multi-agent-v1/")
# CSM = CarlaScenesManager(cpath)
# CDM = CSM.get_scene_dataset_by_name(CSM.splits_scenes["val"][0])
# vid_folder = "videos"

cpath = os.path.join("/data/shared/CARLA/multi-agent-intersection/")
CSM = CarlaScenesManager(cpath)
CDM = CSM.get_scene_dataset_by_name(CSM.splits_scenes["train"][0])
vid_folder = "videos_intersection"

# cpath = "../examples/sim_results"
print(CSM.scenes)
print(f"{len(CDM)} frames")

## Test Different Perception/Tracking Approaches

In [ ]:
# Here we need OpenCOOD. Thankfully, most libraries like pytorch are actually compatible.
# (1) git submodule add -b carla_demo https://github.com/zqzqz/OpenCOOD.git submodules/OpenCOOD (already added in .gitmodules)
# (2) pip install cumm spconv-cu113
# (3) cd submodules/OpenCOOD && python opencood/utils/setup.py build_ext --inplace
# (4) cd submodules/OpenCOOD && python setup.py develop
# (5) Download https://drive.google.com/file/d/11pG0kf2uR9N_o_ACBi_zfd7flGCgZt40/view?usp=sharing into models/
# Folder structure is like models/pointpillar_attentive_fusion/{config.yaml and latest.pth}

import os
import numpy as np
import torch

import opencood.hypes_yaml.yaml_utils as yaml_utils
from opencood.tools import train_utils, inference_utils
from opencood.data_utils.datasets import build_dataset
from opencood.utils import box_utils


class OpencoodPerception:
    def __init__(
        self,
        fusion_method="early",
        model_name="pointpillar",
        model_path="/data/shared/models/opencood/",
    ):
        assert model_name in ["pixor", "voxelnet", "second", "pointpillar"]
        assert fusion_method in ["early", "intermediate", "late"]
        self.name = "{}_{}".format(model_name, fusion_method)
        self.devices = "cuda:0"
        self.model_name = model_name
        self.fusion_method = fusion_method
        self.model_dir = os.path.join(
            model_path,
            "{}_{}_fusion".format(
                self.model_name,
                self.fusion_method
                if self.fusion_method != "intermediate"
                else "attentive",
            ),
        )
        self.config_file = os.path.join(self.model_dir, "config.yaml")

        hypes = yaml_utils.load_yaml(self.config_file, None)
        self.model = train_utils.create_model(hypes)
        self.dataset = build_dataset(hypes, visualize=False, train=False)
        # we assume gpu is necessary
        if torch.cuda.is_available():
            self.model.cuda()
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        _, self.model = train_utils.load_saved_model(self.model_dir, self.model)
        self.model.eval()

        self.preprocessors = {
            "early": self.dataset.preprocess_from_carla,
            "intermediate": self.dataset.preprocess_from_carla,
            "late": self.dataset.preprocess_from_carla,
        }
        self.inference_processors = {
            "early": inference_utils.inference_early_fusion,
            "intermediate": inference_utils.inference_intermediate_fusion,
            "late": inference_utils.inference_late_fusion,
        }

    def run(self, multi_vehicle_case, ego_id):
        batch = self.preprocessors[self.fusion_method](multi_vehicle_case, ego_id)
        batch_data = self.dataset.collate_batch_test([batch])
        with torch.no_grad():
            batch_data = train_utils.to_device(batch_data, self.device)
            pred_box_tensor, pred_score, gt_box_tensor = self.inference_processors[
                self.fusion_method
            ](batch_data, self.model, self.dataset)
        if pred_box_tensor is None:
            pred_bboxes = np.array([])
            pred_scores = np.array([])
        else:
            pred_bboxes = pred_box_tensor.cpu().numpy()
            pred_bboxes = box_utils.corner_to_center(pred_bboxes, order="lwh")
            pred_bboxes[:, 2] -= 0.5 * pred_bboxes[:, 5]
            pred_scores = pred_score.cpu().numpy()
        return pred_bboxes, pred_scores

In [ ]:
from avstack.geometry import (
    Sphere,
    GlobalOrigin3D,
    transform_orientation,
    Box3D,
    Position,
    Attitude,
)

from avstack.modules.perception.detections import BoxDetection
from avstack.modules.perception.object3d import MMDetObjectDetector3D
from avstack.modules.tracking.tracker3d import BasicBoxTracker3D
from avstack.modules.tracking.multisensor import MeasurementBasedMultiTracker

from avstack.datastructs import DataContainer

# init models
agents = list(range(len(CDM.get_agents(frame=1))))
agent_is_static = {
    i: "static" in CDM.get_agent(frame=1, agent=i).obj_type for i in agents
}
percep_veh = MMDetObjectDetector3D(model="pointpillars", dataset="carla-vehicle")
percep_inf = MMDetObjectDetector3D(model="pointpillars", dataset="carla-infrastructure")
percep_col = OpencoodPerception(model_name="pointpillar", fusion_method="intermediate")
trackers = {agent: BasicBoxTracker3D() for agent in agents}
trackers["central"] = MeasurementBasedMultiTracker(tracker=BasicBoxTracker3D())
trackers["collab"] = BasicBoxTracker3D()

# init data structures
dets = {}
tracks = {}
imgs_all = {agent: [] for agent in agents}
pcs_all = {agent: [] for agent in agents}
dets_all = {agent: [] for agent in agents}
dets_all["collab"] = []
tracks_all = {agent: [] for agent in agents}
tracks_all["central"] = []
tracks_all["collab"] = []
agent_0_frames = CDM.get_frames(sensor="lidar-0", agent=0)[1:-1]
platforms_all = {agent: [] for agent in agents}

# flags for this run
run_distributed_perception = True
run_distributed_tracking = True
run_centralized_tracking = True
run_collaborative_perception = True
run_collaborative_tracking = True

# run loop
n_frames_max = 60
ego_agent = agents[0]
for frame in tqdm(agent_0_frames[: min(n_frames_max, len(agent_0_frames))]):
    found_data = False
    fovs = {}
    platforms = {}
    perception_input = {}
    for agent in agents:
        ###############################################
        # GET DATA
        ###############################################
        lidar_sensor = "lidar-0"
        camera_sensor = "camera-0"
        img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
        pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
        imgs_all[agent].append(img)
        pcs_all[agent].append(pc)
        objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)
        calib = CDM.get_calibration(frame=frame, sensor=lidar_sensor, agent=agent)
        fovs[agent] = Sphere(radius=100)
        platforms[agent] = calib.reference
        platforms_all[agent].append(calib.reference)

        ###############################################
        # DISTRIBUTED PERCEPTION
        ###############################################
        found_data = True
        if run_distributed_perception:
            if agent_is_static[agent]:
                dets[agent] = percep_inf(pc)
            else:
                dets[agent] = percep_veh(pc)
            dets_all[agent].append(dets[agent])

        ###############################################
        # DISTRIBUTED TRACKING USING DISTRIBUTED PERCEP
        ###############################################
        if run_distributed_tracking:
            assert run_distributed_perception
            tracks[agent] = trackers[agent](dets[agent], platform=calib.reference)
            if not isinstance(tracks[agent], DataContainer):
                raise
            tracks_all[agent].append([track.box3d.copy() for track in tracks[agent]])

        ###############################################
        # PREP FOR COLLABORATIVE PERCEPTION
        ###############################################
        # Construct the input data structure for collaborative perception model.
        ref_global = pc.calibration.reference.integrate(start_at=GlobalOrigin3D)
        position = ref_global.x
        attitude = ref_global.q
        euler = transform_orientation(attitude, "quat", "euler")
        lidar_pose = np.asarray(
            [
                *position,
                np.degrees(euler[0]),
                np.degrees(euler[2]),
                np.degrees(euler[1]),
            ]
        )
        perception_input[agent] = {
            "lidar": pc.data.x,
            # This pose is in OPV2V format (x, y, z, roll, yaw, pitch) and the rotation are in degrees not radians.
            "lidar_pose": lidar_pose,
        }

    ###############################################
    # COLLABORATIVE PERCEPTION/TRACKING
    ###############################################
    if run_collaborative_perception:
        pred_bboxes, _ = percep_col.run(perception_input, ego_agent)
        collab_dets = []
        for pred_bbox in pred_bboxes:
            det = BoxDetection(
                "lidar-0",
                Box3D(
                    position=Position(x=pred_bbox[0:3], reference=platforms[ego_agent]),
                    # Note that Carla coordination has a reverted y axis (?). Taking a negation of yaw seems make it right.
                    attitude=Attitude(
                        q=transform_orientation([0, 0, -pred_bbox[6]], "euler", "quat"),
                        reference=platforms[ego_agent],
                    ),
                    hwl=pred_bbox[3:6][::-1],
                ),
                platforms[ego_agent],
            )
            collab_dets.append(det)
        collab_dets = DataContainer(
            data=collab_dets, frame=frame, timestamp=0, source_identifier="collab"
        )
        dets_all["collab"].append(collab_dets)

    if run_collaborative_tracking:
        tracks["collab"] = trackers["collab"](
            detections=collab_dets,
            platform=platforms[ego_agent],
        )
        tracks_all["collab"].append(tracks["collab"])

    ###############################################
    # CENTRALIZED TRACKING USING DISTRIBUTED PERCEP
    ###############################################
    # run central tracker on all detections
    if found_data:
        if run_centralized_tracking:
            tracks["central"] = trackers["central"](
                detections=dets,
                fovs=fovs,
                platforms=platforms,
            )
            tracks_all["central"].append(tracks["central"])

#### Verify that the union of the two is similar to the central

In [ ]:
# from avapi.visualize.snapshot import show_lidar_bev_with_boxes, show_boxes_bev

# # extent = [[-70, 70], [-70, 70], [-100, 100]]
# extent = [[-50, 200], [-200, 50], [-100, 100]]

# show_boxes_bev(boxes=tracks["central"])
# show_lidar_bev_with_boxes(pc=pc, boxes=tracks["central"], extent=extent)

In [ ]:
from utils import make_agent_movies, make_central_movie, make_collab_movie

# agent-specific movie
# if run_distributed_tracking:
#     print("Making distributed agent movies")
#     extent = [[-70, 70], [-70, 70], [-100, 100]]
#     for agent in agents:
#         make_agent_movies(
#             imgs=imgs_all[agent],
#             pcs=pcs_all[agent],
#             dets=dets_all[agent],
#             tracks=tracks_all[agent],
#             agent=agent,
#             extent=extent,
#             percep_movies=False,
#             track_movies=True,
#             img_movies=True,
#             bev_movies=True,
#             vid_folder=vid_folder,
#         )


# central tracking movie
if run_centralized_tracking:
    print("Making centralized tracking movies")
    extent = [[-150, 20], [-80, 40], [-100, 100]]
    make_central_movie(
        pcs_all,
        tracks_all["central"],
        ego=platforms_all[ego_agent],
        extent=extent,
        vid_folder=vid_folder,
        colormethod="channel-4",
    )


# central perception movie
if run_collaborative_tracking:
    print("Making collaborative tracking movies")
    make_collab_movie(
        pcs_all,
        tracks_all["collab"],
        ego=platforms_all[ego_agent],
        extent=extent,
        vid_folder=vid_folder,
        colormethod="channel-4",
    )